## OLD w/ Prior concatenated

In [ ]:
# # OLD ---- adds prior to train
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.metrics import roc_auc_score, log_loss, average_precision_score
# import lightgbm as lgb
# from catboost import CatBoostClassifier
# # import xgboost as xgb
# import ast

# # ── 1. Load ──────────────────────────────────────────────────────────────────
# # train_df = pd.read_csv('Train.csv')
# # test_df  = pd.read_csv('Test.csv')
# train_df = pd.read_csv('/app/digicow/data/Train.csv')
# prior_df = pd.read_csv('/app/digicow/data/Prior.csv')
# test_df = pd.read_csv('/app/digicow/data/Test.csv')

# # train_df = pd.concat([prior_df, train_df], ignore_index=True) # PRIOR

# # raise SystemExit("Stopping here to check data.")

# # ── 2. Parse topics ──────────────────────────────────────────────────────────
# def parse_topics(s):
#     try:
#         parsed = ast.literal_eval(s)
#         if isinstance(parsed, list):
#             flat = []
#             for item in parsed:
#                 if isinstance(item, list): flat.extend(item)
#                 else: flat.append(item)
#             return flat
#     except:
#         return []
        
# full_train = pd.concat([prior_df, train_df], ignore_index=True)

# mlb = MultiLabelBinarizer()
# topics_train = pd.DataFrame(
#     # mlb.fit_transform(train_df['topics_list'].apply(parse_topics)),
#     # columns=mlb.classes_, index=train_df.index
#     mlb.fit_transform(full_train['topics_list'].apply(parse_topics)),
#     columns=mlb.classes_, index=full_train.index
# )
# topics_test = pd.DataFrame(
#     mlb.transform(test_df['topics_list'].apply(parse_topics)),
#     columns=mlb.classes_, index=test_df.index
# )
# # train_df = pd.concat([train_df, topics_train], axis=1).drop(columns=['topics_list'])
# full_train = pd.concat([full_train, topics_train], axis=1).drop(columns=['topics_list'])
# test_df  = pd.concat([test_df,  topics_test],  axis=1).drop(columns=['topics_list'])

# # raise SystemExit("Stopping here to check data.")

# # # ── 3. Temporal split ────────────────────────────────────────────────────────
# # train_df['dt'] = pd.to_datetime(train_df['training_day'])
# # test_df['dt']  = pd.to_datetime(test_df['training_day'])
# # cutoff = pd.Timestamp('2025-03-01')

# # df_tr  = train_df[train_df['dt'] < cutoff].copy()
# # df_val = train_df[train_df['dt'] >= cutoff].copy()
# # print(f"Train: {len(df_tr):,} | Val: {len(df_val):,}")

# # ── 3. Temporal split ────────────────────────────────────────────────────────
# # train_df['dt'] = pd.to_datetime(train_df['training_day'])
# full_train['dt'] = pd.to_datetime(full_train['training_day'])
# test_df['dt']  = pd.to_datetime(test_df['training_day'])
# # prior_df['dt'] = pd.to_datetime(prior_df['training_day'])

# prior_encoded = full_train.iloc[:len(prior_df)].copy()
# train_encoded = full_train.iloc[len(prior_df):].copy()

# # cutoff = pd.Timestamp('2025-03-01')
# # df_val = train_df[train_df['dt'] >= cutoff].copy()
# # df_tr  = pd.concat([prior_df, train_df[train_df['dt'] < cutoff]], ignore_index=True)
# # print(f"Train: {len(df_tr):,} | Val: {len(df_val):,}")

# cutoff = pd.Timestamp('2025-01-01')
# df_val = train_encoded[train_encoded['dt'] >= cutoff].copy()
# df_tr  = pd.concat([prior_encoded, train_encoded[train_encoded['dt'] < cutoff]], ignore_index=True)
# print(f"Train: {len(df_tr):,} | Val: {len(df_val):,}")

# # # ── 3b. Target encoding (computed from df_tr only, no leakage) ────────────
# # targets = ['07', '90', '120']
# # te_cols = []
# # for days in targets:
# #     col = f'adopted_within_{days}_days'
# #     for grp in ['trainer', 'county', 'subcounty', 'ward']:
# #         rate_map    = df_tr.groupby(grp)[col].mean()
# #         global_mean = df_tr[col].mean()
# #         col_name    = f'{grp}_adopt_{days}d'
# #         df_tr[col_name]   = df_tr[grp].map(rate_map).fillna(global_mean)
# #         df_val[col_name]  = df_val[grp].map(rate_map).fillna(global_mean)
# #         test_df[col_name] = test_df[grp].map(rate_map).fillna(global_mean)
# #         te_cols.append(col_name)

# # raise SystemExit("Stopping here to check data.")
# # ── 4. Features ──────────────────────────────────────────────────────────────
# cat_cols     = ['gender', 'registration', 'age', 'trainer', 'county', 'subcounty', 'ward']
# topic_cols   = list(mlb.classes_)
# numeric_cols = ['belong_to_cooperative', 'has_topic_trained_on'] + topic_cols #+ te_cols
# feature_cols = cat_cols + numeric_cols

# # LightGBM needs categoricals as 'category' dtype
# def prep_lgb(df):
#     X = df[feature_cols].copy()
#     for c in cat_cols:
#         X[c] = X[c].astype('category')
#     return X

# X_tr_lgb   = prep_lgb(df_tr)
# X_val_lgb  = prep_lgb(df_val)
# X_test_lgb = prep_lgb(test_df)

# # CatBoost takes raw strings — no encoding needed
# X_tr_cat   = df_tr[feature_cols].fillna('missing')
# X_val_cat  = df_val[feature_cols].fillna('missing')
# X_test_cat = test_df[feature_cols].fillna('missing')

# cat_feature_indices = [feature_cols.index(c) for c in cat_cols]

# targets = ['07', '90', '120']
# lgb_preds  = {}
# cat_preds  = {}
# # xgb_preds = {}
# ens_preds  = {}

# # ── 5. Train per target ──────────────────────────────────────────────────────
# for days in targets:
#     col = f'adopted_within_{days}_days'
#     y_tr  = df_tr[col].values
#     y_val = df_val[col].values

#     prevalence = y_tr.mean()
#     scale_pos  = (1 - prevalence) / prevalence  # for imbalance

#     # ── LightGBM ──
#     lgb_tr  = lgb.Dataset(X_tr_lgb,  label=y_tr)
#     lgb_val = lgb.Dataset(X_val_lgb, label=y_val, reference=lgb_tr)

#     params = {
#         'objective':        'binary',
#         'metric':           ['binary_logloss', 'auc'],
#         'scale_pos_weight': scale_pos,
#         'learning_rate':    0.05,
#         'num_leaves':       31,
#         'min_child_samples': 20,
#         'feature_fraction': 0.8,
#         'bagging_fraction': 0.8,
#         'bagging_freq':     5,
#         'verbose':          -1,
#         'seed':             42,
#     }
#     lgb_model = lgb.train(
#         params,
#         lgb_tr,
#         num_boost_round=500,
#         valid_sets=[lgb_val],
#         callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
#     )
#     lgb_val_prob  = lgb_model.predict(X_val_lgb)
#     lgb_test_prob = lgb_model.predict(X_test_lgb)
#     lgb_preds[days] = lgb_test_prob

#     auc   = roc_auc_score(y_val, lgb_val_prob)
#     ll    = log_loss(y_val, lgb_val_prob)
#     prauc = average_precision_score(y_val, lgb_val_prob)
#     print(f"[LGB {days}d]  AUC={auc:.4f}  LogLoss={ll:.4f}  PR-AUC={prauc:.4f}  Naive-PR={prevalence:.4f}")

#     # ── CatBoost ──
#     cat_model = CatBoostClassifier(
#         iterations=1000,
#         learning_rate=0.03,
#         depth=6,
#         loss_function='Logloss',
#         eval_metric='Logloss',  # optimise for LogLoss (75% of score)
#         l2_leaf_reg=5,
#         cat_features=cat_feature_indices,
#         early_stopping_rounds=50,
#         random_seed=42,
#         verbose=0,
#     )
#     cat_model.fit(
#         X_tr_cat, y_tr,
#         eval_set=(X_val_cat, y_val),
#     )
#     cat_val_prob  = cat_model.predict_proba(X_val_cat)[:, 1]
#     cat_test_prob = cat_model.predict_proba(X_test_cat)[:, 1]
#     cat_preds[days] = cat_test_prob

#     auc   = roc_auc_score(y_val, cat_val_prob)
#     ll    = log_loss(y_val, cat_val_prob)
#     prauc = average_precision_score(y_val, cat_val_prob)
#     print(f"[CAT {days}d]  AUC={auc:.4f}  LogLoss={ll:.4f}  PR-AUC={prauc:.4f}")

#     # # ── XGBoost ──
#     # xgb_tr  = xgb.DMatrix(X_tr_lgb,  label=y_tr,  enable_categorical=True)
#     # xgb_val = xgb.DMatrix(X_val_lgb, label=y_val, enable_categorical=True)
#     # xgb_test = xgb.DMatrix(X_test_lgb,             enable_categorical=True)

#     # xgb_params = {
#     #     'objective':        'binary:logistic',
#     #     'eval_metric':      ['logloss', 'auc'],
#     #     'scale_pos_weight': scale_pos,
#     #     'learning_rate':    0.05,
#     #     'max_depth':        4,
#     #     'subsample':        0.8,
#     #     'colsample_bytree': 0.8,
#     #     'min_child_weight': 20,
#     #     'tree_method':      'hist',
#     #     'device':           'cpu',
#     #     'seed':             42,
#     # }
#     # xgb_model = xgb.train(
#     #     xgb_params,
#     #     xgb_tr,
#     #     num_boost_round=500,
#     #     evals=[(xgb_val, 'val')],
#     #     early_stopping_rounds=50,
#     #     verbose_eval=False,
#     # )
#     # xgb_val_prob  = xgb_model.predict(xgb_val)
#     # xgb_test_prob = xgb_model.predict(xgb_test)
#     # xgb_preds[days] = xgb_test_prob

#     # auc   = roc_auc_score(y_val, xgb_val_prob)
#     # ll    = log_loss(y_val, xgb_val_prob)
#     # prauc = average_precision_score(y_val, xgb_val_prob)
#     # print(f"[XGB {days}d]  AUC={auc:.4f}  LogLoss={ll:.4f}  PR-AUC={prauc:.4f}")


    
#     # ── Ensemble (simple average) ──
#     ens_val_prob  = (lgb_val_prob + cat_val_prob) / 2
#     ens_test_prob = (lgb_test_prob + cat_test_prob) / 2
#     ens_preds[days] = ens_test_prob

#     auc   = roc_auc_score(y_val, ens_val_prob)
#     ll    = log_loss(y_val, ens_val_prob)
#     prauc = average_precision_score(y_val, ens_val_prob)
#     print(f"[ENS {days}d]  AUC={auc:.4f}  LogLoss={ll:.4f}  PR-AUC={prauc:.4f}")
#     print()

#     # # ── Ensemble (3-way average) ──
#     # ens_val_prob  = (lgb_val_prob + cat_val_prob + xgb_val_prob) / 3
#     # ens_test_prob = (lgb_test_prob + cat_test_prob + xgb_test_prob) / 3
#     # ens_preds[days] = ens_test_prob

#     # auc   = roc_auc_score(y_val, ens_val_prob)
#     # ll    = log_loss(y_val, ens_val_prob)
#     # prauc = average_precision_score(y_val, ens_val_prob)
#     # print(f"[ENS {days}d]  AUC={auc:.4f}  LogLoss={ll:.4f}  PR-AUC={prauc:.4f}")
#     # print()

# # # ── 6. Submission ────────────────────────────────────────────────────────────
# # ss = pd.read_csv('SampleSubmission.csv')[['ID']]
# # for days in targets:
# #     ss[f'Target_{days}_AUC']     = ens_preds[days]
# #     ss[f'Target_{days}_LogLoss'] = ens_preds[days]

# # col_order = ['ID',
# #              'Target_07_AUC', 'Target_90_AUC', 'Target_120_AUC',
# #              'Target_07_LogLoss', 'Target_90_LogLoss', 'Target_120_LogLoss']
# # ss[col_order].to_csv('submission_ensemble.csv', index=False)
# # print("Saved submission_ensemble.csv")

Train: 56,200 | Val: 2,218
[LGB 07d]  AUC=0.9559  LogLoss=0.0614  PR-AUC=0.2416  Naive-PR=0.0140
[CAT 07d]  AUC=0.9427  LogLoss=0.0542  PR-AUC=0.2755
[ENS 07d]  AUC=0.9566  LogLoss=0.0532  PR-AUC=0.2601

[LGB 90d]  AUC=0.9321  LogLoss=0.0779  PR-AUC=0.1605  Naive-PR=0.0304
[CAT 90d]  AUC=0.9518  LogLoss=0.0566  PR-AUC=0.3277
[ENS 90d]  AUC=0.9532  LogLoss=0.0640  PR-AUC=0.2799

[LGB 120d]  AUC=0.9084  LogLoss=0.0883  PR-AUC=0.0823  Naive-PR=0.0419
[CAT 120d]  AUC=0.9167  LogLoss=0.0637  PR-AUC=0.2977
[ENS 120d]  AUC=0.9364  LogLoss=0.0739  PR-AUC=0.2973



In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_auc_score, log_loss, average_precision_score
import lightgbm as lgb
from catboost import CatBoostClassifier
import optuna
import ast
import json

optuna.logging.set_verbosity(optuna.logging.WARNING)

# ── 1. Load ───────────────────────────────────────────────────────────────────
train_df = pd.read_csv('/app/digicow/data/Train.csv')
prior_df = pd.read_csv('/app/digicow/data/Prior.csv')
test_df  = pd.read_csv('/app/digicow/data/Test.csv')

# ── 2. Parse topics ───────────────────────────────────────────────────────────
def parse_topics(s):
    try:
        parsed = ast.literal_eval(s)
        if isinstance(parsed, list):
            flat = []
            for item in parsed:
                if isinstance(item, list): flat.extend(item)
                else: flat.append(item)
            return flat
    except:
        return []

full_train = pd.concat([prior_df, train_df], ignore_index=True)

mlb = MultiLabelBinarizer()
topics_train = pd.DataFrame(
    mlb.fit_transform(full_train['topics_list'].apply(parse_topics)),
    columns=mlb.classes_, index=full_train.index
)
topics_test = pd.DataFrame(
    mlb.transform(test_df['topics_list'].apply(parse_topics)),
    columns=mlb.classes_, index=test_df.index
)
full_train = pd.concat([full_train, topics_train], axis=1).drop(columns=['topics_list'])
test_df    = pd.concat([test_df, topics_test],     axis=1).drop(columns=['topics_list'])

# ── 3. Temporal split ─────────────────────────────────────────────────────────
full_train['dt'] = pd.to_datetime(full_train['training_day'])
test_df['dt']    = pd.to_datetime(test_df['training_day'])

prior_encoded = full_train.iloc[:len(prior_df)].copy()
train_encoded = full_train.iloc[len(prior_df):].copy()

cutoff = pd.Timestamp('2025-03-01')
df_val = train_encoded[train_encoded['dt'] >= cutoff].copy()
df_tr  = pd.concat([prior_encoded, train_encoded[train_encoded['dt'] < cutoff]], ignore_index=True)
print(f"Train: {len(df_tr):,} | Val: {len(df_val):,}")

# ── 4. Features ───────────────────────────────────────────────────────────────
cat_cols     = ['gender', 'registration', 'age', 'trainer', 'county', 'subcounty', 'ward']
topic_cols   = list(mlb.classes_)
numeric_cols = ['belong_to_cooperative', 'has_topic_trained_on'] + topic_cols
feature_cols = cat_cols + numeric_cols

def prep_lgb(df):
    X = df[feature_cols].copy()
    for c in cat_cols:
        X[c] = X[c].astype('category')
    return X

X_tr_lgb   = prep_lgb(df_tr)
X_val_lgb  = prep_lgb(df_val)
X_test_lgb = prep_lgb(test_df)

X_tr_cat   = df_tr[feature_cols].fillna('missing')
X_val_cat  = df_val[feature_cols].fillna('missing')
X_test_cat = test_df[feature_cols].fillna('missing')

cat_feature_indices = [feature_cols.index(c) for c in cat_cols]
targets = ['07', '90', '120']

# ── 5. Optuna tuning ──────────────────────────────────────────────────────────
def tune_lgb(X_tr, y_tr, X_val, y_val, scale_pos):
    def objective(trial):
        params = {
            'objective':         'binary',
            'metric':            'binary_logloss',
            'scale_pos_weight':  scale_pos,
            'verbose':           -1,
            'seed':              42,
            'learning_rate':     trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'num_leaves':        trial.suggest_int('num_leaves', 16, 128),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
            'feature_fraction':  trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq':      trial.suggest_int('bagging_freq', 1, 10),
            'reg_alpha':         trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
            'reg_lambda':        trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
        }
        ds_tr  = lgb.Dataset(X_tr, label=y_tr)
        ds_val = lgb.Dataset(X_val, label=y_val, reference=ds_tr)
        model = lgb.train(
            params, ds_tr,
            num_boost_round=500,
            valid_sets=[ds_val],
            callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
        )
        return log_loss(y_val, model.predict(X_val))
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50, show_progress_bar=True)
    return study.best_params

def tune_cat(X_tr, y_tr, X_val, y_val, cat_feature_indices):
    def objective(trial):
        model = CatBoostClassifier(
            iterations=1000, random_seed=42, verbose=0,
            cat_features=cat_feature_indices,
            early_stopping_rounds=50,
            loss_function='Logloss', eval_metric='Logloss',
            learning_rate=      trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            depth=              trial.suggest_int('depth', 4, 8),
            l2_leaf_reg=        trial.suggest_float('l2_leaf_reg', 1.0, 20.0),
            bagging_temperature=trial.suggest_float('bagging_temperature', 0.0, 2.0),
            random_strength=    trial.suggest_float('random_strength', 0.0, 2.0),
            border_count=       trial.suggest_int('border_count', 32, 255),
        )
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
        return log_loss(y_val, model.predict_proba(X_val)[:, 1])
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30, show_progress_bar=True)
    return study.best_params

def optimize_weight(cat_prob, lgb_prob, y_true):
    def objective(trial):
        w = trial.suggest_float('cat_weight', 0.0, 1.0)
        return log_loss(y_true, w * cat_prob + (1-w) * lgb_prob)
    study = optuna.create_study(direction='minimize',
        sampler=optuna.samplers.TPESampler(n_startup_trials=20, seed=42))
    study.optimize(objective, n_trials=100)
    return study.best_params['cat_weight']

best_params  = {}
best_weights = {}

for days in targets:
    col = f'adopted_within_{days}_days'
    y_tr  = df_tr[col].values
    y_val = df_val[col].values
    prevalence = y_tr.mean()
    scale_pos  = (1 - prevalence) / prevalence

    print(f"\n── Tuning {days}d ──")
    lgb_best = tune_lgb(X_tr_lgb, y_tr, X_val_lgb, y_val, scale_pos)
    cat_best = tune_cat(X_tr_cat, y_tr, X_val_cat, y_val, cat_feature_indices)
    best_params[days] = {'lgb': lgb_best, 'cat': cat_best}

    # retrain to get val probs for weight tuning
    ds_tr  = lgb.Dataset(X_tr_lgb, label=y_tr)
    ds_val = lgb.Dataset(X_val_lgb, label=y_val, reference=ds_tr)
    lgb_m  = lgb.train({'objective':'binary','metric':'binary_logloss',
                         'scale_pos_weight':scale_pos,'verbose':-1,'seed':42,**lgb_best},
                        ds_tr, num_boost_round=500, valid_sets=[ds_val],
                        callbacks=[lgb.early_stopping(50,verbose=False),lgb.log_evaluation(0)])
    cat_m  = CatBoostClassifier(iterations=1000, loss_function='Logloss',
                                 cat_features=cat_feature_indices, early_stopping_rounds=50,
                                 random_seed=42, verbose=0, **cat_best)
    cat_m.fit(X_tr_cat, y_tr, eval_set=(X_val_cat, y_val))

    best_weights[days] = optimize_weight(cat_m.predict_proba(X_val_cat)[:,1],
                                          lgb_m.predict(X_val_lgb), y_val)
    print(f"CAT weight: {best_weights[days]:.3f}")

# Save
with open('/app/digicow/best_params_concat.json', 'w') as f:
    json.dump({'params': best_params, 'weights': best_weights}, f, indent=2)
print("Saved best_params_concat.json")

# Load
with open('/app/digicow/best_params_concat.json') as f:
    saved = json.load(f)
best_params  = saved['params']
best_weights = saved['weights']

# ── 6. Train with tuned params ────────────────────────────────────────────────
lgb_preds = {}
cat_preds = {}
ens_preds = {}

for days in targets:
    col = f'adopted_within_{days}_days'
    y_tr  = df_tr[col].values
    y_val = df_val[col].values
    prevalence = y_tr.mean()
    scale_pos  = (1 - prevalence) / prevalence

    # ── LightGBM ──
    lgb_tr     = lgb.Dataset(X_tr_lgb, label=y_tr)
    lgb_val_ds = lgb.Dataset(X_val_lgb, label=y_val, reference=lgb_tr)
    params = {
        'objective': 'binary', 'metric': ['binary_logloss', 'auc'],
        'scale_pos_weight': scale_pos, 'verbose': -1, 'seed': 42,
        **best_params[days]['lgb']
    }
    lgb_model = lgb.train(
        params, lgb_tr,
        num_boost_round=500,
        valid_sets=[lgb_val_ds],
        callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
    )
    lgb_val_prob  = lgb_model.predict(X_val_lgb)
    lgb_test_prob = lgb_model.predict(X_test_lgb)
    lgb_preds[days] = lgb_test_prob
    print(f"[LGB {days}d]  AUC={roc_auc_score(y_val, lgb_val_prob):.4f}  LogLoss={log_loss(y_val, lgb_val_prob):.4f}")

    # ── CatBoost ──
    cat_model = CatBoostClassifier(
        iterations=1000, loss_function='Logloss', eval_metric='Logloss',
        cat_features=cat_feature_indices, early_stopping_rounds=50,
        random_seed=42, verbose=0,
        **best_params[days]['cat']
    )
    cat_model.fit(X_tr_cat, y_tr, eval_set=(X_val_cat, y_val))
    cat_val_prob  = cat_model.predict_proba(X_val_cat)[:, 1]
    cat_test_prob = cat_model.predict_proba(X_test_cat)[:, 1]
    cat_preds[days] = cat_test_prob
    print(f"[CAT {days}d]  AUC={roc_auc_score(y_val, cat_val_prob):.4f}  LogLoss={log_loss(y_val, cat_val_prob):.4f}")

    # ── Ensemble ──
    # ens_val_prob  = (lgb_val_prob + cat_val_prob) / 2
    # ens_test_prob = (lgb_test_prob + cat_test_prob) / 2
    w = best_weights[days]
    ens_val_prob  = w * cat_val_prob  + (1-w) * lgb_val_prob
    ens_test_prob = w * cat_test_prob + (1-w) * lgb_test_prob
    ens_preds[days] = ens_test_prob
    print(f"[ENS {days}d]  AUC={roc_auc_score(y_val, ens_val_prob):.4f}  LogLoss={log_loss(y_val, ens_val_prob):.4f}\n")

# # ── 7. Submission ─────────────────────────────────────────────────────────────
# ss = pd.read_csv('/app/digicow/data/SampleSubmission.csv')[['ID']]
# for days in targets:
#     ss[f'Target_{days}_AUC']     = ens_preds[days]
#     ss[f'Target_{days}_LogLoss'] = ens_preds[days]

# col_order = ['ID',
#              'Target_07_AUC', 'Target_90_AUC', 'Target_120_AUC',
#              'Target_07_LogLoss', 'Target_90_LogLoss', 'Target_120_LogLoss']
# ss[col_order].to_csv('/app/digicow/submission_tuned.csv', index=False)
# print("Saved submission_tuned.csv")

Train: 57,180 | Val: 1,238

── Tuning 07d ──


Best trial: 15. Best value: 0.0343982: 100%|██████████| 30/30 [24:52<00:00, 49.73s/it] 


CAT weight: 1.000

── Tuning 90d ──


Best trial: 8. Best value: 0.0381926: 100%|██████████| 30/30 [16:17<00:00, 32.58s/it]


CAT weight: 1.000

── Tuning 120d ──


Best trial: 28. Best value: 0.0440553: 100%|██████████| 30/30 [14:17<00:00, 28.60s/it]


CAT weight: 0.389
Saved best_params_concat.json
[LGB 07d]  AUC=0.9761  LogLoss=0.0494
[CAT 07d]  AUC=0.9844  LogLoss=0.0344
[ENS 07d]  AUC=0.9844  LogLoss=0.0344

[LGB 90d]  AUC=0.9706  LogLoss=0.0669
[CAT 90d]  AUC=0.9784  LogLoss=0.0382
[ENS 90d]  AUC=0.9784  LogLoss=0.0382

[LGB 120d]  AUC=0.9726  LogLoss=0.0816
[CAT 120d]  AUC=0.9661  LogLoss=0.0441
[ENS 120d]  AUC=0.9763  LogLoss=0.0622



In [3]:
# ── Submission ────────────────────────────────────────────────────────────────
ss = pd.read_csv('/app/digicow/data/SampleSubmission.csv')[['ID']]

for days in targets:
    ss[f'Target_{days}_AUC']     = ens_preds[days]
    ss[f'Target_{days}_LogLoss'] = ens_preds[days]

col_order = ['ID',
             'Target_07_AUC', 'Target_90_AUC', 'Target_120_AUC',
             'Target_07_LogLoss', 'Target_90_LogLoss', 'Target_120_LogLoss']

ss = ss[col_order]
ss.to_csv('/app/digicow/submission.csv', index=False)
print(f"Saved submission.csv — {ss.shape[0]} rows")
print(ss.head(3))

Saved submission.csv — 5621 rows
          ID  Target_07_AUC  Target_90_AUC  Target_120_AUC  Target_07_LogLoss  \
0  ID_LEG1GM       0.000646       0.004792        0.094164           0.000646   
1  ID_1UKOKW       0.000265       0.000858        0.012638           0.000265   
2  ID_U5H2YK       0.299165       0.153741        0.248571           0.299165   

   Target_90_LogLoss  Target_120_LogLoss  
0           0.004792            0.094164  
1           0.000858            0.012638  
2           0.153741            0.248571  
